In [90]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

nav = webdriver.Chrome()

tabela_produtos = pd.read_excel("./buscas.xlsx")

display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [91]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    # analisar se ele não tem nenhum termo banido
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    # analisar se ele tem TODOS os termos do nome do produto
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
    
    return tem_todos_termos_produtos

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    lista_ofertas = []

    # pesquisando pelo produto no google
    nav.get("https://www.google.com/")
    nav.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea').send_keys(produto, Keys.ENTER)

    # entrar na aba shopping
    elemento = nav.find_element("xpath", '/html/body/div[4]/div/div[5]/div/div[1]/div/div/div[1]/div/div/div/div/div[1]/div[2]/a')
    elemento.click()

    # pegando as informações dos produtos
    lista_resultados = nav.find_elements("class name", "i0X6df")

    for resultado in lista_resultados:
        nome = resultado.find_element("class name", "tAxDx").text
        nome = nome.lower()

        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

        # selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produtos = True
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element("class name", "a8Pemb").text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)

            # se o preco está entre preco_minimo e preco_maximo
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element('class name', 'bONr3b')
                elemento_pai = elemento_referencia.find_element('xpath', '..') #pegando o elemento que vem logo antes dele ('pai')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome, preco, link))
    
    return lista_ofertas

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    lista_ofertas = []

    nav.get("https://www.buscape.com.br/")
    nav.find_element("xpath", '/html/body/div[1]/main/header/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

    time.sleep(2)
    lista_resultados = nav.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')

    for resultado in lista_resultados:
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text
        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text
        nome = nome.lower()
        link = resultado.get_attribute('href')

        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)

            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas.append((nome, preco, link))
    
    return lista_ofertas


In [92]:
produto = "iphone 12 64 gb"
termos_banidos = "mini watch"
preco_minimo = 2000
preco_maximo = 4500

lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
display(lista_ofertas_google_shopping)

lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
display(lista_ofertas_buscape)


[('vitrine iphone 12 preto 64gb',
  2272.39,
  'https://www.google.com/url?url=https://www.carrefour.com.br/vitrine--iphone--12--preto-64gb-mp931849769/p&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwiHwYK94ryFAxV1rpUCHQEHDfwQgOUECPcL&usg=AOvVaw33TeCggyclNOtDodDM3MeI'),
 ('apple iphone 12 (64 gb) - azul',
  2559.2,
  'https://www.google.com/url?url=https://www.horizonplay.com.br/apple/iphone/apple-iphone-12-64gb-azul-novo-lacrado-tela-super-retina-xdr-oled-6-1%3Fvariant_id%3D21013%26parceiro%3D8926%26srsltid%3DAfmBOoruImKk2UIu5KYP0KeRCtLgTKRo-dYrCA-Z5bPjPXQzJCCNdJ2nD-A&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwiHwYK94ryFAxV1rpUCHQEHDfwQguUECIoM&usg=AOvVaw1SVBJo55diVCIcxuVst8RE'),
 ('vitrine. iphone 12 branco 64gb',
  2272.39,
  'https://www.google.com/url?url=https://www.carrefour.com.br/vitrine-iphone--12--branco--64gb-mp931605276/p&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwiHwYK94ryFAxV1rpUCHQEHDfwQgOUECJ4M&usg=AOvVaw1YfiZakA4xLfc4Iy6h4H4F'),
 ('usado: iphone 12 64gb preto muito

[('smartphone apple iphone 12 64gb câmera dupla',
  2766.66,
  'https://www.buscape.com.br/celular/smartphone-apple-iphone-12-64gb-ios?_lc=88&searchterm=iphone%2012%2064%20gb'),
 ('smartphone apple iphone 12 vermelho 64gb câmera dupla',
  3599.0,
  'https://www.buscape.com.br/celular/smartphone-apple-iphone-12-vermelho-64gb-ios?_lc=88&searchterm=iphone%2012%2064%20gb'),
 ('smartphone apple iphone 12 vermelho usado 64gb',
  2384.1,
  'https://www.buscape.com.br/celular/smartphone-apple-iphone-12-vermelho-usado-64gb-camera-dupla?_lc=88&searchterm=iphone%2012%2064%20gb')]